In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open('text/WoO.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
vocab_size = len(chars)

Tokeniser

In [ ]:
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

Data Batch Retriever

In [ ]:
training_size = int(0.8*len(data))
train_data = data[:training_size]
test_data = data[training_size:]

block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

#x, y = get_batch("train")


GPT Class Definition

In [ ]:
n_embd = 352 # number of embeddings/ embedding dimension
n_layer = 4 # Number of decoding blocks
# 352 % 4 = 4  <------ important as it allows different features to be taken from different heads

dropout = 0.1 # percentage of neurons to block

class MultiHeadAttention(nn.Module):
    """ Perform parallel self-attention calculations"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # Setup parallel heads
        self.proj = nn.Linear(head_size * num_heads, n_embd) # Project values onto embedding
        self.dropout = nn.Dropout(dropout)  # Prevent overfitting by randomly blocking some neuron activations

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # Concatenate calculated features into single featurelist
        out = self.dropout(self.proj(out)) # Prevent overfitting by randomly blocking some neuron activations
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # Perform Linear Transformation
            nn.ReLU(), # Apply ReLU activation Function (creates non-linearity)
            nn.Linear(4 * n_embd, n_embd), # Perform Linear Transformation
            nn.Dropout(dropout), # Prevent overfitting by randomly blocking some neuron activations
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        #n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head # Number of features captured in 'MultiHeadAttention'
        self.sa = MultiHeadAttention(n_head, head_size) # Self Attention
        self.ffwd = FeedFoward(n_embd) # Generate new value with feed forward
        self.ln1 = nn.LayerNorm(n_embd) # Normalising of values
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x) # Self Attention
        x = self.ln1(x + y) # Normalise input value with self Attention value
        y = self.ffwd(x) # Feed Foward
        x = self.ln2(x + y) # Normalise previous value added with the feed forward value
        return x

class GPT(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # Generates decoding blocks

        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights) # Generate initial parameter weight within a standard deviation of mean to prevent sparatic or stagnant training

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        token_emb = self.token_embedding_table(index) # retrieve token embedding
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # retrieve positional embedding
        x = token_emb + pos_emb # combine embedding values


        x = self.blocks(x) # feed value into decoding blocks
        x = self.ln_f(x) # perform linear transformation
        logits = self.lm_head(x) # convert to probabilities for softmax

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            index_next = torch.multinomial(probs, num_samples=1)

            index = torch.cat((index, index_next), dim=1)
        
        return index
    
model = GPT(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

Loss Estimation Function

In [ ]:
eval_iters=250

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
learning_rate = 3e-4
max_iters=10000
eval_iters=250

optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

In [ ]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = nn.Re